Using `ipyparallel`
====

Parallel execution is tightly integrated with Jupyter in the `ipyparallel` package. Install with

```bash
conda install ipyparallel
ipcluster nbextension enable
```

[Official documentation](https://ipyparallel.readthedocs.org/en/latest/)

In [1]:
import numpy as np

## Starting engines

We will only use engines on local cores which does not require any setup - see [docs](https://ipyparallel.readthedocs.org/en/latest/process.html) for detailed instructions on how to set up a remote cluster, including setting up to use Amazon EC2 clusters.

You can start a cluster on the `IPython Clusters` tab in the main Jupyter browser window or via the command line with

```
ipcluster start -n <put desired number of engines to run here>
```

The main advantage of developing parallel applications using `ipyparallel` is that it can be done interactively within Jupyter.

## Basic concepts of `ipyparallel`

In [2]:
from ipyparallel import Client

The client connects to the cluster of "remote" engines that perfrom the actual computation. These engines may be on the same machine or on a cluster. 

In [3]:
rc = Client()

In [4]:
rc.ids

[0, 1, 2, 3, 4, 5, 6, 7]

A view provides access to a subset of the engines available to the client. Jobs are submitted to the engines via the view. A direct view allows the user to explicitly send work specific engines. The load balanced view is like the `Pool` object in `multiprocessing`, and manages the scheduling and distribution of jobs for you.

**Direct view**

In [5]:
dv = rc[:]

Add 10 sets of 3 numbers in parallel using all engines.

In [6]:
dv.map_sync(lambda x, y, z: x + y + z, range(10), range(10), range(10))

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

Add 10 sets of 3 numbers in parallel using only alternate engines.

In [7]:
rc[::2].map_sync(lambda x, y, z: x + y + z, range(10), range(10), range(10))

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

Add 10 sets of 3 numbers using a specific engine.

In [8]:
rc[2].map_sync(lambda x, y, z: x + y + z, range(10), range(10), range(10))

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

**Load balanced view**

Use this when you have many jobs that take different amounts of time to complete.

In [9]:
lv = rc.load_balanced_view()

In [10]:
lv.map_sync(lambda x: sum(x), np.random.random((10, 100000)))

[50059.03811156327,
 49956.13106095374,
 50058.942016221408,
 50017.9069873673,
 49915.685312931419,
 50033.315794786198,
 49970.957693484197,
 49994.709667674178,
 50037.627611867385,
 50157.999084306517]

#### Calling functions with apply

In contrast to `map`, `apply` is just a simple function call run on all remote engines, and has the usual function signature `apply(f, *args, **kwargs)`. It is a primitive on which other more useful functions (such as `map`) are built upon.

In [11]:
rc[1:3].apply_sync(lambda x, y: x**2 + y**2, 3, 4)

[25, 25]

In [12]:
rc[1:3].apply_sync(lambda x, y: x**2 + y**2, x=3, y=4)

[25, 25]

### Synchronous and asynchronous jobs

We have used the `map_sync` and `apply_sync` methods. The `sync` suffix indicate that we want to run a synchronous job. Synchronous jobs `block` until all the computation is done and return the result.

In [13]:
res = dv.map_sync(lambda x, y, z: x + y + z, range(10), range(10), range(10))

In [14]:
res

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

In contrast, asynchronous jobs return immediately so that you can do other work, but returns a `AsyncMapResult` object, similar to the `future` object returned by the `concurrent.futures` package. You can query its status, cancel running jobs and retrieve results once they have been computed.

In [15]:
res = dv.map_async(lambda x, y, z: x + y + z, range(10), range(10), range(10))

In [16]:
res

<AsyncMapResult: <lambda>:finished>

In [17]:
res.done()

True

In [18]:
res.get()

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

There is also a `map` method that by default uses asynchronous mode, but you can change this by setting the `block` attribute or function argument.

In [19]:
res = dv.map(lambda x, y, z: x + y + z, range(10), range(10), range(10))

In [20]:
res.get()

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

Change blocking mode for just one job.

In [21]:
res = dv.map(lambda x, y, z: x + y + z, range(10), range(10), range(10), block=True)

In [22]:
res

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

Change blocking mode for this view so that all jobs are synchronous.

In [23]:
dv.block = True

In [24]:
res = dv.map(lambda x, y, z: x + y + z, range(10), range(10), range(10))

In [25]:
res

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

### Remote function decorators 

The `@remote` decorator results in functions that will execute simultaneously on all engines in a view. For example, you can use this decorator if you always want to run $n$ independent parallel MCMC chains.

In [26]:
@dv.remote(block = True)
def f1(n):
    import numpy as np
    return np.random.rand(n)

In [27]:
f1(4)

[array([ 0.36391956,  0.37063603,  0.83564952,  0.5211274 ]),
 array([ 0.58601816,  0.12463745,  0.65892738,  0.74891614]),
 array([ 0.95696614,  0.67097947,  0.21674694,  0.73518342]),
 array([ 0.11304809,  0.69523297,  0.66372836,  0.97232926]),
 array([ 0.05428269,  0.69884128,  0.92001545,  0.24408769]),
 array([ 0.48405267,  0.27424395,  0.69997456,  0.16886651]),
 array([ 0.72636556,  0.30889676,  0.34382904,  0.37849331]),
 array([ 0.67376915,  0.98566768,  0.39774162,  0.20274586])]

The @parallel decorator breaks up elementwise operations and distributes them.

In [28]:
@dv.parallel(block = True)
def f2(x):
    return x

In [29]:
f2(range(15))

[range(0, 2),
 range(2, 4),
 range(4, 6),
 range(6, 8),
 range(8, 10),
 range(10, 12),
 range(12, 14),
 range(14, 15)]

In [30]:
@dv.parallel(block = True)
def f3(x):
    return sum(x)

In [31]:
f3(range(15))

[1, 5, 9, 13, 17, 21, 25, 14]

In [32]:
@dv.parallel(block = True)
def f4(x, y):
    return x + y

In [33]:
f4(np.arange(10), np.arange(10))

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18])

####  Example: Use the `@parallel` decorator to speed up Mandelbrot calculations

In [34]:
def mandel1(x, y, max_iters=80):
    c = complex(x, y)
    z = 0.0j
    for i in range(max_iters):
        z = z*z + c
        if z.real*z.real + z.imag*z.imag >= 4:
            return i
    return max_iters

In [35]:
@dv.parallel(block = True)
def mandel2(x, y, max_iters=80):
    c = complex(x, y)
    z = 0.0j
    for i in range(max_iters):
        z = z*z + c
        if z.real*z.real + z.imag*z.imag >= 4:
            return i
    return max_iters

In [36]:
x = np.arange(-2, 1, 0.01)
y = np.arange(-1, 1, 0.01)
X, Y = np.meshgrid(x, y)

In [37]:
%%time
im1 = np.reshape(list(map(mandel1, X.ravel(), Y.ravel())), (len(y), len(x)))

CPU times: user 863 ms, sys: 4.56 ms, total: 867 ms
Wall time: 865 ms


In [38]:
%%time
im2 = np.reshape(mandel2.map(X.ravel(), Y.ravel()),  (len(y), len(x)))

CPU times: user 43.9 ms, sys: 10.5 ms, total: 54.4 ms
Wall time: 253 ms


In [39]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].grid(False)
axes[0].imshow(im1, cmap='jet')
axes[1].grid(False)
axes[1].imshow(im2, cmap='jet')
pass

NameError: name 'plt' is not defined

### Functions with dependencies

Modules imported locally are NOT available in the remote engines.

In [ ]:
import time
import datetime

In [ ]:
def g1(x):
    time.sleep(0.1)
    now = datetime.datetime.now()
    return (now, x)

This fails with an Exception because the `time` and `datetime` modules are not imported in the remote engines.

```python
dv.map_sync(g1, range(10))
```

The simplest fix is to import the module(s) *within* the function

In [ ]:
def g2(x):
    import time, datetime
    time.sleep(0.1)
    now = datetime.datetime.now()
    return (now, x)

In [ ]:
dv.map_sync(g2, range(5))

Alternatively, you can simultaneously import both locally and in the remote engines with the `sync_import` context manager.

In [ ]:
with dv.sync_imports():
    import time
    import datetime

Now the `g1` function will work.

In [ ]:
dv.map_sync(g1, range(5))

Finally, there is also a `require` decorator that can be used. This will force the remote engine to import all packages given.

In [40]:
from ipyparallel import require

In [41]:
@require('scipy.stats')
def g3(x):
    return scipy.stats.norm(0,1).pdf(x)

In [42]:
dv.map(g3, np.arange(-3, 4))

[0.0044318484119380075,
 0.053990966513188063,
 0.24197072451914337,
 0.3989422804014327,
 0.24197072451914337,
 0.053990966513188063,
 0.0044318484119380075]

### Moving data around

We can send data to remote engines with `push` and retrieve them with `pull`, or using the dictionary interface. For example, you can use this to distribute a large lookup table to all engines once instead of repeatedly as a function argument.

In [43]:
dv.push(dict(a=3, b=2))

[None, None, None, None, None, None, None, None]

In [44]:
def f(x):
    global a, b
    return a*x + b

In [45]:
dv.map_sync(f, range(5))

[2, 5, 8, 11, 14]

In [46]:
dv.pull(('a', 'b'))

[[3, 2], [3, 2], [3, 2], [3, 2], [3, 2], [3, 2], [3, 2], [3, 2]]

#### You can also use the dictionary interface as an alternative to push and pull

In [47]:
dv['c'] = 5

In [48]:
dv['a']

[3, 3, 3, 3, 3, 3, 3, 3]

In [49]:
dv['c']

[5, 5, 5, 5, 5, 5, 5, 5]

## Working with compiled code

### Numba

Using `numba.jit` is straightforward.

In [50]:
with dv.sync_imports():
    import numba

importing numba on engine(s)


In [51]:
@numba.jit
def f_numba(x):
    return np.sum(x)

In [52]:
dv.map(f_numba, np.random.random((6, 4)))

[1.727117082710574,
 2.030104113894334,
 2.528579015128492,
 1.971340951890772,
 3.431482055472422,
 1.8739277049802001]

### Cython

We need to do some extra work to make sure the shared libary compiled with cython is available to the remote engines:

- Compile a `named` shared module with the `-n` flag
- Use `np.ndarray[dtype, ndim]` in place of memroy views
    - for example, double[:] becomes np.ndarray[np.float64_t, ndim=1]
- Move the shared library to the `site-packages` directory
    - Cython magic moules can be found in `~/.ipython/cython`
- Import the modules remtoely in the usual ways

In [53]:
%load_ext cython

In [54]:
%%cython -n cylib

import cython
import numpy as np
cimport numpy as np

@cython.boundscheck(False)
@cython.wraparound(False)
def f(np.ndarray[np.float64_t, ndim=1] x):
    x.setflags(write=True)
    cdef int i
    cdef int n = x.shape[0]
    cdef double s = 0

    for i in range(n):
        s += x[i]
    return s

#### Copy the compiled module in `site-packages` so that the remote engines can import it

In [57]:
import os
import glob
import site
import shutil
src = glob.glob(os.path.join(os.path.expanduser('~/'), '.ipython', 'cython', 'cylib*so'))[0]
dst = site.getsitepackages()[0]
shutil.copy(src, dst)

'/Users/cliburn/anaconda2/envs/p3/lib/python3.5/site-packages/cylib.cpython-35m-darwin.so'

In [58]:
with dv.sync_imports():
    import cylib

importing cylib on engine(s)


Using parallel magic commands
----

In practice, most users will simply use the `%px` magic to execute code in parallel from within the notebook. This is the simplest way to use `ipyparallel`.

In [59]:
dv.map(cylib.f, np.random.random((6, 4)))

[2.2402249848386893,
 2.469154138118082,
 2.625243199432881,
 2.7101560978701977,
 2.033144977939152,
 2.0108942456624295]

### %px

This sends the command to all targeted engines.

In [61]:
%px import numpy as np
%px a = np.random.random(4)
%px a.sum()

Out[0:4]: 1.5827323235275481

Out[1:4]: 1.9745635147041902

Out[2:4]: 1.2147373109361463

Out[3:4]: 2.2126544598732161

Out[4:4]: 1.4743638091542319

Out[5:4]: 1.4264247964212471

Out[6:4]: 2.4712987180472314

Out[7:4]: 1.4125339510243891

#### List comprehensions in parallel

The `scatter` method partitions and distributes data to all engines. The `gather` method does the reverse. Together with `%px`, we can simulate parallel list comprehensions.

In [62]:
dv.scatter('a', np.random.randint(0, 10, 10))
%px print(a)

[stdout:0] [6 0]
[stdout:1] [9 9]
[stdout:2] [7]
[stdout:3] [8]
[stdout:4] [7]
[stdout:5] [3]
[stdout:6] [0]
[stdout:7] [9]


In [63]:
dv.gather('a')

array([6, 0, 9, 9, 7, 8, 7, 3, 0, 9])

In [64]:
dv.scatter('xs', range(24))
%px y = [x**2 for x in xs]
np.array(dv.gather('y'))

array([  0,   1,   4,   9,  16,  25,  36,  49,  64,  81, 100, 121, 144,
       169, 196, 225, 256, 289, 324, 361, 400, 441, 484, 529])

#### Running magic functions in parallel

In [65]:
%%px --target [1,3]
%matplotlib inline
import seaborn as sns
x = np.random.normal(np.random.randint(-10, 10), 1, 100)
sns.kdeplot(x);

CompositeError: one or more exceptions from call to method: execute
[1:execute]: TypeError: slice indices must be integers or None or have an __index__ method
[3:execute]: TypeError: slice indices must be integers or None or have an __index__ method

### Running in non-blocking mode

In [66]:
%%px --target [1,3] --noblock
%matplotlib inline
import seaborn as sns
x = np.random.normal(np.random.randint(-10, 10), 1, 100)
sns.kdeplot(x);

<AsyncResult: execute>

In [67]:
%pxresult

CompositeError: one or more exceptions from call to method: execute
[1:execute]: TypeError: slice indices must be integers or None or have an __index__ method
[3:execute]: TypeError: slice indices must be integers or None or have an __index__ method